In [1]:
# run initial imports
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from scipy.integrate import odeint
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

from fbprophet import Prophet

from ipywidgets import widgets

from fbprophet.plot import plot_plotly
import plotly.offline as py

from datetime import datetime

In [2]:
NOW = datetime.now()

In [3]:
#setup initial data frames
country = 'us'
input_dir = './data/countries/' + country + '/'
us_cases = pd.read_csv(input_dir + 'covid_19_'+ country +'_cases.csv')
us_totals = pd.read_csv(input_dir + './covid_19_' + country + '_totals.csv')

FileNotFoundError: [Errno 2] File ./data/countries/us/covid_19_us_cases.csv does not exist: './data/countries/us/covid_19_us_cases.csv'

In [ ]:
#google docs from covidtracking.com
#us_ct = pd.read_csv('http://covidtracking.com/api/states.csv')

In [ ]:
#correct date parsing on some of the JH data
us_cases['Date'] = pd.to_datetime(us_cases['Date'])
us_totals['Date'] = pd.to_datetime(us_totals['Date'])

In [ ]:
# some basic metadata here
US_pop = 330435890
US_tests = 58536
us_totals['Active Pop PCT'] = us_totals['Active Cases'] / US_pop
US_tr = US_tests / US_pop
# per capita test rate in china
CHINA_pop = 1408526449
CHINA_pop_GD = 113500000
CHINA_tests_GD = 320000
CHINA_tr = CHINA_tests_GD / CHINA_pop_GD
print('CN Test Rate:', CHINA_tr)
print('US Test Rate:', US_tr)

In [ ]:
us_totals['Likely Cases C86'].max()

In [ ]:
#display(covid_19_ts[covid_19_ts['Country/Region'] == 'US'])

In [ ]:
us_totals['Total Deaths'].max() / us_totals['Total Confirmed'].max() 

In [ ]:
#display(us_cases)

In [ ]:
#display(covid_19_by_date)

In [ ]:
us_chart = px.bar(us_cases[us_cases['Date'] >= '2020-02-20'], 
    x="Date", y="Confirmed", color="Province/State", title="Confirmed US Cases")
us_chart.show()

In [ ]:
us_chart = px.bar(us_cases[us_cases['Date'] >= '2020-02-20'], 
    x="Date", y="Likely Cases 1.8pct", color="Province/State", title="Likely US Cases")
us_chart.show()

In [ ]:
us_chart = px.bar(us_totals[us_totals['Date'] >= '2020-02-20'], 
    x="Date", y="Active Cases", color="Death Rate", 
    title="Active US Active Cases (confirmed - recovered - dead) - Current as of: "+str(NOW), 
    )
us_chart.show()

In [ ]:
us_chart = px.bar(us_totals[us_totals['Date'] >= '2020-02-20'], 
    x="Date", y="Total Deaths", color="Active Cases", 
    title="Total US Deaths (Confirmed->Dead) - Current as of: "+str(NOW), 
    )
us_chart.show()

In [ ]:
us_chart = px.bar(us_totals[us_totals['Date'] >= '2020-02-20'], 
    x="Date", y="Likely Cases 1.8pct", color="Death Rate", 
    title="Likely US Active Cases (confirmed - recovered - dead) - Current as of: "+str(NOW), 
    )
us_chart.show()

In [ ]:
us_totals['New Active Cases'] = us_totals['Active Cases'] - us_totals['Active Cases'].shift()
us_totals['New Cases'] = us_totals['Confirmed'] - us_totals['Confirmed'].shift()
us_totals['New Case PCT Change'] = us_totals['Confirmed'].pct_change()

In [ ]:
us_chart = px.bar(us_totals[us_totals['Date'] >= '2020-02-20'], 
    x="Date", y="New Case PCT Change", color="Death Rate", 
    title="New US Active Cases (confirmed - recovered - dead) - Current as of: "+str(NOW), 
    )
us_chart.show()

In [ ]:
us_chart = px.bar(us_totals[us_totals['Date'] >= '2020-03-03'], 
    x="Date", y="Death Change Rate", color="New Case PCT Change", 
    title="New US Deaths (confirmed->dead) - Current as of: "+str(NOW), 
    )
us_chart.show()

In [ ]:
def exponenial_func(x, a, b, c):
    return a*np.exp(-b*x)+c

us_totals=us_totals.sort_values('Date')
us_totals=us_totals.reset_index()
us_totals['Day'] = us_totals.index
x=us_totals.index
y=us_totals['Active Cases']

popt, pcov = curve_fit(exponenial_func, x, y, p0=(1, 1e-6, 1.25))

In [ ]:
fDay=3
xx = np.linspace(0, len(y)+fDay)
yy = exponenial_func(xx, *popt)
z=2.79248349e-04 * np.exp(3.11496172e-01 * x) + 1.03712386e+01

In [ ]:
popt # 2.79248349e-04 * np.exp(-3.11496172e-01 * x) + 1.03712386e+01

In [ ]:
dateAx = []
valAx = []
foreDates = []
startDate = us_totals['Date'].min()
for i in xx:
    d = np.round(i,0)
    foreDates.append((startDate + pd.DateOffset(days=d)).date())
    if d%7 == 0:
        valAx.append(d)
        dateAx.append((startDate + pd.DateOffset(days=d)).date())

In [ ]:
from sklearn.metrics import r2_score

R2Val = str(np.round(r2_score(y, z), 3))

In [ ]:
trace1 = go.Scatter(
                  x=x,
                  y=y,
                  mode='markers',
                  name='Active Cases'
                  )

trace2 = go.Scatter(
                  x=xx,
                  y=yy,
                  mode='lines',
                  name='Forecast'
                  )

trace3 = go.Scatter(
                  x=x,
                  y=z,
                  mode='lines',
                  name='Test Fit'
                  )

layout = go.Layout(
    title='Active Cases by Date, Actual + ' + str(fDay) + ' day Forecast as of: '+str(NOW)+'<br />R2: '+R2Val,
    titlefont=dict(
        family='Arial, sans-serif',
        size=24,
        color='black'
    ),
    xaxis=dict(
        tickvals=valAx,
        ticktext=dateAx,
        showticklabels=True,
        ticks='outside',
        tickangle=45,
        tickfont=dict(
            family='Arial, sans-serif',
            size=14,
            color='black'
        )
    ),
    yaxis=dict(
        title='Active Cases',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='grey'
        ),
        ticks='outside',
        showticklabels=True,
        tickfont=dict(
            family='Arial, sans-serif',
            size=14,
            color='black'
        ) #,type='log'
    )
)
    
fig = go.Figure(layout=layout)
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.show()

In [ ]:
def exponenial_func(x, a, b, c):
    return a*np.exp(-b*x)+c

us_totals=us_totals.sort_values('Date')
us_totals=us_totals.reset_index(drop=True)
us_totals['Day'] = us_totals.index
x=us_totals.index
y=us_totals['Likely Cases 1.8pct']

popt, pcov = curve_fit(exponenial_func, x, y, p0=(1, 1e-6, 1.25))

xx = np.linspace(0, len(y)+fDay)
yy = exponenial_func(xx, *popt)
z=2.79248349e-04 * np.exp(3.11496172e-01 * x) + 1.03712386e+01

dateAx = []
valAx = []
foreDates = []
startDate = us_totals['Date'].min()
for i in xx:
    d = np.round(i,0)
    foreDates.append((startDate + pd.DateOffset(days=d)).date())
    if d%7 == 0:
        valAx.append(d)
        dateAx.append((startDate + pd.DateOffset(days=d)).date())

R2Val = str(np.round(r2_score(y, z), 3))

trace1 = go.Scatter(
                  x=x,
                  y=y,
                  mode='markers',
                  name='Active Cases'
                  )

trace2 = go.Scatter(
                  x=xx,
                  y=yy,
                  mode='lines',
                  name='Forecast'
                  )

trace3 = go.Scatter(
                  x=x,
                  y=z,
                  mode='lines',
                  name='Test Fit'
                  )

layout = go.Layout(
    title='Likely Cases 1.8pct by Date, Actual + '+str(fDay)+' day Forecast as of: '+str(NOW)+'<br />R2: '+R2Val,
    titlefont=dict(
        family='Arial, sans-serif',
        size=24,
        color='black'
    ),
    xaxis=dict(
        tickvals=valAx,
        ticktext=dateAx,
        showticklabels=True,
        ticks='outside',
        tickangle=45,
        tickfont=dict(
            family='Arial, sans-serif',
            size=14,
            color='black'
        )
    ),
    yaxis=dict(
        title='Active Cases',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='grey'
        ),
        ticks='outside',
        showticklabels=True,
        tickfont=dict(
            family='Arial, sans-serif',
            size=14,
            color='black'
        ) #,type='log'
    )
)
    
fig = go.Figure(layout=layout)
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.show()

In [ ]:
# now let's run the forecast with fbprophet
pDay=3
fb_df = us_totals[['Date', 'Active Cases']].copy()
fb_df = fb_df.sort_values('Date').reset_index(drop=True)
fb_df.columns = ['ds','y']
#print(fb_df)

m = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False, growth='linear')
m.fit(fb_df)

future = m.make_future_dataframe(periods=pDay)

forecast = m.predict(future)

py.init_notebook_mode()

fig = plot_plotly(m, forecast, xlabel='Date', ylabel='Active Cases', uncertainty=True, figsize=(1400,900))  # This returns a plotly Figure
fig.update_layout(title='Active '+country+' COVID-19 Cases and Forecast ('+str(pDay)+' day)')
py.iplot(fig)

In [ ]:
# see here for some good parameter tuning
N = US_pop * 0.2
T=60
I0, R0 = us_totals['Likely Cases 1.8pct'].max(), us_totals['Recovered'].max()*120
S0 = N - I0 - R0
beta, gamma = 0.3, 1/36
t = np.linspace(0, T, T)
y0 = S0, I0, R0

#SIR Deriv
def deriv(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

ret = odeint(deriv, y0, t, args=(N, beta, gamma))
S, I, R = ret.T

D = (I/1000000)*0.018
R = (R/1000000) - D

fig = plt.figure(figsize=(24,7), facecolor='w')
ax = fig.add_subplot(111, axisbelow=True)
ax.plot(t, S/1000000, 'b', alpha=0.5, lw=2, label='Susceptible')
ax.plot(t, I/1000000, 'r', alpha=0.5, lw=2, label='Infected')
ax.plot(t, D, 'y', alpha=0.5, lw=2, label='Dead')
ax.plot(t, R, 'g', alpha=0.5, lw=2, label='Recovered with immunity')
ax.set_xlabel('Time /days')
ax.set_title('US - SIR+d model based on known disease characteristics and response at: '+str(NOW) + 
            '\nTotal Infected: '+'{:,.2f}'.format(int(np.max(S)))+
            '\nTotal Recovered: '+'{:,.2f}'.format(int(np.max(R)*1000000))+
            '\nTotal Dead: '+'{:,.2f}'.format(int(np.max(D)*1000000)),
            loc='left'
            )
#ax.set_ylabel('Number (1000s)')
#ax.set_ylim(0,1.2)
ax.yaxis.set_tick_params(length=0)
ax.xaxis.set_tick_params(length=0)
ax.grid(b=True, which='major', c='w', lw=2, ls='-')
legend = ax.legend()
legend.get_frame().set_alpha(0.5)
for spine in ('top', 'right', 'bottom', 'left'):
    ax.spines[spine].set_visible(False)
plt.show()

In [ ]:
N = US_pop * 0.2
T=180
I0, R0 = us_totals['Likely Cases 1.8pct'].max(), us_totals['Recovered'].max()*120
S0 = N - I0 - R0
beta, gamma = 0.1, 1/28
t = np.linspace(0, T, T)
y0 = S0, I0, R0

#SIR Deriv
def deriv(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

ret = odeint(deriv, y0, t, args=(N, beta, gamma))
S, I, R = ret.T

D = (I/1000000)*0.008
R = (R/1000000) - D

fig = plt.figure(figsize=(24,7), facecolor='w')
ax = fig.add_subplot(111, axisbelow=True)
ax.plot(t, S/1000000, 'b', alpha=0.5, lw=2, label='Susceptible')
ax.plot(t, I/1000000, 'r', alpha=0.5, lw=2, label='Infected')
ax.plot(t, D, 'y', alpha=0.5, lw=2, label='Dead')
ax.plot(t, R, 'g', alpha=0.5, lw=2, label='Recovered with immunity')
ax.set_xlabel('Time /days')
ax.set_title('US - SIR+d model based on known disease characteristics and AGGRESIVE response at: '+str(NOW) + 
            '\nTotal Infected: '+'{:,.2f}'.format(int(np.max(S)))+
            '\nTotal Recovered: '+'{:,.2f}'.format(int(np.max(R)*1000000))+
            '\nTotal Dead: '+'{:,.2f}'.format(int(np.max(D)*1000000)),
            loc='left'
            )
#ax.set_ylabel('Number (1000s)')
#ax.set_ylim(0,1.2)
ax.yaxis.set_tick_params(length=0)
ax.xaxis.set_tick_params(length=0)
ax.grid(b=True, which='major', c='w', lw=2, ls='-')
legend = ax.legend()
legend.get_frame().set_alpha(0.5)
for spine in ('top', 'right', 'bottom', 'left'):
    ax.spines[spine].set_visible(False)
plt.show()